# Finetune GAP-text2SQL for SQL Query Generation on Bank Account Fraud Dataset Suite

[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/vxcent/sagemaker_gap_demo/blob/main/rat-sql-gap/notebook.ipynb) 

### Citations & Credits:

* #### Original inference code and model are from [AAAI 2021 paper - Learning Contextual Representations for Semantic Parsing with Generation-Augmented Pre-Training](https://arxiv.org/abs/2012.10309)
* #### The Bank Fraud Dataset (security_1) is from [Kaggle - Bank Account Fraud Dataset Suite (NeurIPS 2022)](https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022?resource=download)

## Step 0. Install all necessary packages

### Open up a Terminal shell through Sagemaker Studio Lab for issuing the following commands

In [ ]:
conda activate default
conda install -y python=3.7
conda install -y -c conda-forge jsonnet openjdk
conda install -y pytorch=1.5 cudatoolkit=10.2 -c pytorch

### Run the following cells to install Python dependencies 
* Running the command in terminal shell is recommended to keep track of progress

In [ ]:
%pip install -r requirements.txt

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
mkdir third_party
wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
unzip stanford-corenlp-full-2018-10-05.zip -d third_party/


### Download the Pretrained Model and Finetune Checkpoint (through terminal shell)

#### Download Finetune Checkpoint

### Sanity Check: Make sure we're in the rat-sql-gap directory

In [ ]:
cd sagemaker_gap_demo/rat-sql-gap

In [ ]:
mkdir -p logdir/bart_run_1/bs\=12\,lr\=1.0e-04\,bert_lr\=1.0e-05\,end_lr\=0e0\,att\=1/
curl https://gap-text2sql-public.s3.amazonaws.com/checkpoint-artifacts/gap-finetuned-checkpoint -o logdir/bart_run_1/bs\=12\,lr\=1.0e-04\,bert_lr\=1.0e-05\,end_lr\=0e0\,att\=1/model_checkpoint-00041000

#### Download the Pretrained Model

In [ ]:
mkdir -p pretrained_checkpoint
curl https://gap-text2sql-public.s3.amazonaws.com/checkpoint-artifacts/pretrained-checkpoint -o pretrained_checkpoint/pytorch_model.bin

### How that we've finished the preparations, let's get to running the actual code!

In [4]:
import json
import os
import _jsonnet

In [5]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [6]:
import torch

#### The gap-run.jsonnet file includes metadata used for experiment tracking
#### The most important field `model_config_args` includes hyperperameters used for inferences

#### To learn more about the usage of the 'facebook/bart-large' model, checkout the [Huggin Face 🤗 Documentation](https://huggingface.co/facebook/bart-large) 

* Jsonnet file is taken in to return a json string for subsequent usage

In [8]:
exp_config = json.loads(
    _jsonnet.evaluate_file(
        "experiments/spider-configs/gap-run.jsonnet"))


In [ ]:
exp_config

In [9]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

In [10]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

##### Before we do the actual text-to-SQL inference, we fine-tune the model's `encoder` to learn the SQL DB's schema structure
* ##### The `encoder` takes an input sequence (an English sentence in this case) and processes it into a fixed-size representation, often referred to as a context vector or latent representation.

* ##### The `decode` takes the context vector produced by the encoder and generates an output sequence (e.g., a SQL query). It generates the output sequence step by step, often autoregressively, by using the previously generated tokens and the context vector.

In [11]:
infer_config["model"]["encoder_preproc"]["db_path"] = "data/sqlite_files/"

In [ ]:
infer_config["model"]

In [12]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


In [14]:
model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
checkpoint_step = exp_config["eval_steps"][0]

In [ ]:
print(model_dir)
print(checkpoint_step)

In [ ]:
model = inferer.load_model(model_dir, checkpoint_step)

In [19]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [20]:
db_id = "security_1"

In [21]:
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [22]:
from seq2struct.utils.api_utils import refine_schema_names

In [ ]:
my_schema

In [34]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

* ##### We inspect if there is any Primary Key and Foreign Key relationship in the database schema

In [35]:
schema.keys()

dict_keys(['security_1'])

In [36]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

##### We're using the Stanford CoreNLP module to preprocess the schema items
* That's why we needed OpenJDK as part of the dependencies when setting up the environment

In [37]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [38]:
spider_schema = dataset.schemas[db_id]

It's worth mentioning that the GAP framework also adopted the `Transformers` architecture to achieve what we're testing today

For deeper understanding of the Transformers Architecture used by GAP, I recommend studying [Attention Is All You Need" by Vaswani et al.](https://arxiv.org/abs/1706.03762) published in 2017. It has since become one of the most influential and widely used architectures in the field of natural language processing (NLP) and machine learning. 

### Here's a summary of what's going to happen next:

![Inference Process](inference_process.png)

* #### Given an English question/utterance and a Database schema, GAP MODEL takes all the input and concatenate into a data collection called the `data_item`
* #### The inputs goes through a Masked Language Model process to identify words that are "interesting" and substute those word tokens with a `<mask>`
* #### With the 12-layer transformer, each token in the input can be encoded as contextual representations. For different learning objectives, the representations are utilized by different decoders: 
* Column Prediction(CPred): Capture the alignment between the question(utterance) and the database schema
* Column Recovery (CRec): Discover the connections between the cell values and the column names, thereby infer the right column name from a cell value mentioned
* SQL Generation (GenSQL): Compose complex SQL that requires logical reasoning, generating sophisticated SQL queries

In [39]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [ ]:
code = infer("What type of device operating systems are used by the customer?")
print(code)

In [ ]:
code = infer("Can you provide a breakdown of the different payment types used by customers in the database, and how many customers used each payment type in the last month?")
print(code)

In [ ]:
code = infer("What is the average credit risk score based on age group?")
print(code)

### Let's see some inferences ran on other database schemas!


In [ ]:
db_id = "scholar"
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [ ]:
my_schema

In [45]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [46]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [47]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [ ]:
spider_schema = dataset.schemas[db_id]

In [ ]:
code = infer("How many papers are about deep learning ?")
print(code)

In [ ]:
code = infer("List all academic papers on machine networks for one shot learning")
print(code)